In [1]:
import pandas as pd
import numpy as np

# For plotting the graphs
import matplotlib.pyplot as plt
import seaborn as sns
import csv

# Machine Learning Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Machine Learning Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.neural_network import MLPClassifier

# Machine Learning Model Evaluation
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc, confusion_matrix, classification_report

from sklearn.preprocessing import label_binarize
from itertools import cycle
import polars as pl
import gc
from sklearn.feature_selection import VarianceThreshold
import os
import time


In [2]:
csv_file_path = 'data/final_dataset.csv/final_dataset.csv'
parquet_file_path = 'data/final_dataset.csv/final_dataset.parquet'
try:
    pl.scan_csv(csv_file_path).sink_parquet(parquet_file_path)
except Exception as e:
    print(f"Error converting CSV to Parquet: {e}")
    exit()

In [3]:
import pyarrow.parquet as pq
parquet_file = pq.ParquetFile(parquet_file_path)
columns = parquet_file.schema.names

In [4]:
print(columns)

['', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg'

In [ ]:
# drop columns with zero variance
selector = VarianceThreshold()
selector.fit(pl.read_parquet(parquet_file_path).to_pandas()[columns[:-1]])
columns_to_keep = [columns[i] for i in range(len(columns)-1) if selector.get_support()[i]]
columns_to_keep.append(columns[-1])  # add the label column back